In [ ]:
import torch
import local_llm as lllm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

assets_dir = r"C:/Users/Cameron.Webster/Python/local-llm/assets/bert-base-local1"

encoder = lllm.build_bert_input_encoder(assets_dir, max_len=256, lowercase=True)

texts = ["This is a test.", "Another example."]
encoded = [encoder.encode(t) for t in texts]

input_ids = torch.tensor([e.input_ids for e in encoded], device=device)
attention_mask = torch.tensor([e.attention_mask for e in encoded], device=device)
token_type_ids = torch.tensor([e.token_type_ids for e in encoded], device=device)

num_labels = 8
model = lllm.BertTextClassifier.from_pretrained(
    assets_dir,
    num_labels=num_labels,
    pooling="cls",
)
model.to(device)
model.train()

labels = torch.randint(0, num_labels, (input_ids.size(0),), device=device)

out = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    token_type_ids=token_type_ids,
    labels=labels,
)
loss = out["loss"]
loss.backward()
